In [ ]:
import os
import base64
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from dotenv import load_dotenv
load_dotenv()

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", "https://anildwa-aoai-resource-eastus.openai.azure.com/")
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-5")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2025-01-01-preview")


token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

print("AOAI ENDPOINT:", endpoint)
print("AOAI DEPLOYMENT:", deployment)
print("AOAI API VERSION:", api_version)

client = AzureOpenAI(
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
    api_version="2025-01-01-preview",
)



system_message = """

You are a PostgreSQL AGE query generator. Your job is to produce correct, executable SQL that embeds Cypher for a CRM knowledge graph named customer_graph.

Only output code (one SQL statement per answer) unless asked otherwise.


Graph schema (labels, relationships, properties)

Node labels

Customer — properties are stored under payload:
payload.id, payload.name, payload.segment, payload.owner, payload.satisfaction_score, payload.health, payload.growth_potential, payload.current_arr, payload.current_mrr, payload.timezone, payload.notes

Contract — payload.id, payload.customer_id, payload.start_date, payload.end_date, payload.amount, payload.status, payload.auto_renew, payload.renewal_term_months, payload.last_renewal_date, payload.next_renewal_date

SupportCase — payload.id, payload.customer_id, payload.opened_at, payload.last_updated_at, payload.status, payload.priority, payload.escalation_level, payload.sla_breached, payload.product_area, payload.subject, payload.tags

Communication — payload.id, payload.customer_id, payload.timestamp, payload.channel, payload.counterpart, payload.direction, payload.sentiment, payload.summary

Opportunity — payload.id, payload.customer_id, payload.opp_type, payload.product, payload.stage, payload.amount, payload.opened_at, payload.expected_close

TelemetryMonth — payload.customer_id, payload.month, payload.dau, payload.mau, payload.feature_adoption, payload.usage_hours, payload.incidents

QBRArtifact — payload.customer_id, payload.report_period, payload.highlights, payload.risks, payload.asks, payload.attachments

Product(name), Feature(name) (catalog nodes)

Relationships

(:Customer)-[:ADOPTED_PRODUCT]->(:Product)

(:Customer)-[:HAS_CONTRACT]->(:Contract)

(:Customer)-[:RAISED_CASE]->(:SupportCase)

(:SupportCase)-[:ABOUT_AREA]->(:Feature)

(:Customer)-[:HAD_COMM]->(:Communication)

(:Customer)-[:HAS_OPPORTUNITY]->(:Opportunity)

(:Opportunity)-[:FOR_PRODUCT]->(:Product)

(:Customer)-[:HAS_TELEMETRY]->(:TelemetryMonth)

(:TelemetryMonth)-[:ADOPTED_FEATURE {percent, month}]->(:Feature)

(:Customer)-[:HAS_QBR]->(:QBRArtifact)

All business properties live under .payload. Access them as alias.payload.<field>.

Output format (SQL wrapper)

Always wrap Cypher in this shape and ensure the number of RETURN items equals the column list:
SELECT *
FROM ag_catalog.cypher('customer_graph', $$

  // Cypher goes here

$$) AS (
  col1 ag_catalog.agtype,
  col2 ag_catalog.agtype,
  -- etc.
);

Required conventions & gotchas

Use .payload
Example: c.payload.name, not c.name.

Prefer OPTIONAL MATCH for edges that might be absent. Keep the Customer row even when no matches exist.

Aggregation pattern (AGE-safe)

Do NOT use reduce(...) or pattern/list comprehensions with property access in the filter ([x IN list WHERE x.payload.foo]).

Instead, compute booleans and aggregate with SUM(CASE ...).

To build lists of maps, use:

collect(CASE WHEN cond THEN { ... } ELSE NULL END) AS tmp_list
WITH [x IN tmp_list WHERE x IS NOT NULL] AS clean_list


(Filter only on x IS NOT NULL, not on x.payload....)

Null safety
Wrap numeric aggregations in coalesce(sum(...), 0) and scalar fields in coalesce(field, default) as appropriate.

Case checks
AGE doesn’t support SQL’s lower(). If normalization is required, use toLower(field) = 'value' or compare with explicit ORs (e.g., 'active' OR 'Active' OR 'ACTIVE'). Prefer exact known casing when possible.

Booleans
Use true/false (lowercase). Example: coalesce(x.payload.sla_breached, false).

IDs
Use id(n) for internal IDs. If you need the business ID, use n.payload.id.

Close everything
Close maps } and the $$ block before the AS (...) column definition.

Required conventions & gotchas

Use .payload
Example: c.payload.name, not c.name.

Prefer OPTIONAL MATCH for edges that might be absent. Keep the Customer row even when no matches exist.

Aggregation pattern (AGE-safe)

Do NOT use reduce(...) or pattern/list comprehensions with property access in the filter ([x IN list WHERE x.payload.foo]).

Instead, compute booleans and aggregate with SUM(CASE ...).

To build lists of maps, use:

collect(CASE WHEN cond THEN { ... } ELSE NULL END) AS tmp_list
WITH [x IN tmp_list WHERE x IS NOT NULL] AS clean_list


(Filter only on x IS NOT NULL, not on x.payload....)

Null safety
Wrap numeric aggregations in coalesce(sum(...), 0) and scalar fields in coalesce(field, default) as appropriate.

Case checks
AGE doesn’t support SQL’s lower(). If normalization is required, use toLower(field) = 'value' or compare with explicit ORs (e.g., 'active' OR 'Active' OR 'ACTIVE'). Prefer exact known casing when possible.

Booleans
Use true/false (lowercase). Example: coalesce(x.payload.sla_breached, false).

IDs
Use id(n) for internal IDs. If you need the business ID, use n.payload.id.

Close everything
Close maps } and the $$ block before the AS (...) column definition.

Response rules

Always return a valid SQL statement that executes in PostgreSQL with AGE.

Ensure column aliases in RETURN match the AS ( ... ) column list (names and counts).

Prefer single statement answers.

Use parameters like $customer_name only if the user provided them; otherwise use literals from the question.

If asked for multiple sections (e.g., revenue + cases + opportunities), compute them in one Cypher with proper WITH pipelines and return all requested fields in one row per customer unless a list is explicitly requested.

Hard “don’ts”

Do NOT use reduce(...), list/pattern comprehensions with property access in filters, APOC procedures, or SQL functions like lower() inside Cypher.

Do NOT return a single map while declaring multiple columns (or vice versa).

Do NOT omit closing braces or the $$/AS (...) wrapper.
"""


#system_message = " you are an ai assisant."

chat_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": system_message
            }
        ]
    }
]

def generate_cypher_query(user_input: str) -> str:
    print("using deployment:", deployment)
    print("AOAI ENDPOINT:", endpoint)
    chat_prompt.append({
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": user_input
            }
        ]
    })
    response = client.chat.completions.create(
        model=deployment,
        messages=chat_prompt,
        max_completion_tokens=10000,
        stop=None,

    )
    cypher_query = response.choices[0].message.content
    return cypher_query

AOAI ENDPOINT: https://anildwa-aoai-resource-eastus.openai.azure.com/
AOAI DEPLOYMENT: gpt-5
AOAI API VERSION: 2025-01-01-preview


In [4]:
from age_api_test.pg_age_helper import PGAgeHelper
import os
from dotenv import load_dotenv
load_dotenv()

DSN = dict(
    host=os.getenv("PGHOST", "localhost"),
    port=int(os.getenv("PGPORT", "5432")),
    dbname=os.getenv("PGDATABASE", "postgres"),
    user=os.getenv("PGUSER", "postgres"),
    password=os.getenv("PGPASSWORD", "postgres"),
)

GRAPH = os.getenv("GRAPH", "customer_graph")
test_query = """

SELECT *
FROM ag_catalog.cypher('customer_graph', $$
 
  MATCH (n)
  WHERE id(n) = 1407374883553360
    OPTIONAL MATCH (n)-[e]->(t)
    RETURN
                id(n)            AS id,
                labels(n)        AS label,
                properties(n)    AS properties,
                'edge'           AS kind,
                id(n)            AS src,
                id(t)            AS dst
$$) AS (
 id ag_catalog.agtype,
                label ag_catalog.agtype,
                properties ag_catalog.agtype,
                kind ag_catalog.agtype,
                src ag_catalog.agtype,
                dst ag_catalog.agtype
);

"""


test_query_2 = """

SELECT *
FROM ag_catalog.cypher('customer_graph', $$

  MATCH (c:Customer)
  WHERE c.payload.name = 'Customer 080'
  OPTIONAL MATCH (c)-[:RAISED_CASE]->(sc:SupportCase)
  WITH c, sc, toLower(coalesce(sc.payload.status, '')) AS status_l, toLower(coalesce(sc.payload.priority, '')) AS prio_l
  WITH
    c,
    SUM(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) THEN 1 ELSE 0 END) AS open_case_count,
    SUM(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) AND prio_l = 'low' THEN 1 ELSE 0 END) AS open_low_count,
    SUM(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) AND prio_l = 'medium' THEN 1 ELSE 0 END) AS open_medium_count,
    SUM(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) AND prio_l = 'high' THEN 1 ELSE 0 END) AS open_high_count,
    SUM(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) AND prio_l = 'critical' THEN 1 ELSE 0 END) AS open_critical_count,
    SUM(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) AND coalesce(sc.payload.sla_breached, false) = true THEN 1 ELSE 0 END) AS open_sla_breached_count,
    SUM(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) AND coalesce(sc.payload.escalation_level, 0) > 0 THEN 1 ELSE 0 END) AS open_escalated_count,
    collect(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) THEN {
      id: sc.payload.id,
      subject: sc.payload.subject,
      status: sc.payload.status,
      priority: sc.payload.priority,
      opened_at: sc.payload.opened_at,
      last_updated_at: sc.payload.last_updated_at,
      product_area: sc.payload.product_area,
      escalation_level: coalesce(sc.payload.escalation_level, 0),
      sla_breached: coalesce(sc.payload.sla_breached, false),
      tags: sc.payload.tags
    } ELSE NULL END) AS oc_tmp,
    max(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) THEN sc.payload.last_updated_at ELSE NULL END) AS last_open_case_updated_at,
    min(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) THEN sc.payload.opened_at ELSE NULL END) AS oldest_open_case_opened_at
  WITH
    c,
    coalesce(open_case_count, 0) AS open_case_count,
    coalesce(open_low_count, 0) AS open_low_count,
    coalesce(open_medium_count, 0) AS open_medium_count,
    coalesce(open_high_count, 0) AS open_high_count,
    coalesce(open_critical_count, 0) AS open_critical_count,
    coalesce(open_sla_breached_count, 0) AS open_sla_breached_count,
    coalesce(open_escalated_count, 0) AS open_escalated_count,
    [x IN oc_tmp WHERE x IS NOT NULL] AS open_cases,
    last_open_case_updated_at,
    oldest_open_case_opened_at
  RETURN
    c.payload.id AS customer_id,
    c.payload.name AS customer_name,
    c.payload.owner AS owner,
    c.payload.segment AS segment,
    c.payload.health AS health,
    c.payload.satisfaction_score AS satisfaction_score,
    open_case_count,
    open_low_count,
    open_medium_count,
    open_high_count,
    open_critical_count,
    open_sla_breached_count,
    open_escalated_count,
    last_open_case_updated_at,
    oldest_open_case_opened_at,
    open_cases

$$) AS (
  customer_id ag_catalog.agtype,
  customer_name ag_catalog.agtype,
  owner ag_catalog.agtype,
  segment ag_catalog.agtype,
  health ag_catalog.agtype,
  satisfaction_score ag_catalog.agtype,
  open_case_count ag_catalog.agtype,
  open_low_count ag_catalog.agtype,
  open_medium_count ag_catalog.agtype,
  open_high_count ag_catalog.agtype,
  open_critical_count ag_catalog.agtype,
  open_sla_breached_count ag_catalog.agtype,
  open_escalated_count ag_catalog.agtype,
  last_open_case_updated_at ag_catalog.agtype,
  oldest_open_case_opened_at ag_catalog.agtype,
  open_cases ag_catalog.agtype
);


"""

helper = await PGAgeHelper.create(DSN, GRAPH)


rows = await helper.query_using_sql_cypher(test_query_2)
print(rows)


Executing query:
 

SELECT *
FROM ag_catalog.cypher('customer_graph', $$

  MATCH (c:Customer)
  WHERE c.payload.name = 'Customer 080'
  OPTIONAL MATCH (c)-[:RAISED_CASE]->(sc:SupportCase)
  WITH c, sc, toLower(coalesce(sc.payload.status, '')) AS status_l, toLower(coalesce(sc.payload.priority, '')) AS prio_l
  WITH
    c,
    SUM(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) THEN 1 ELSE 0 END) AS open_case_count,
    SUM(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) AND prio_l = 'low' THEN 1 ELSE 0 END) AS open_low_count,
    SUM(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) AND prio_l = 'medium' THEN 1 ELSE 0 END) AS open_medium_count,
    SUM(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) AND prio_l = 'high' THEN 1 ELSE 0 END) AS open_high_count,
    SUM(CASE WHEN sc IS NOT NULL AND NOT (status_l IN ['closed','resolved']) AND prio_l = 'critical' THEN 1 ELSE 0 END) AS open_critical_count,
    SU

In [3]:
user_input = """

I’m going on a sales call with customer 'Customer 080'” → Provide a consolidated customer insight including:

- Pending cases / open issues

"""
cypher_query = generate_cypher_query(user_input)



rows = await helper.query_using_sql_cypher(cypher_query)
print(rows)


using deployment: gpt-5
AOAI ENDPOINT: https://anildwa-aoai-resource-eastus.openai.azure.com/
Executing query:
 SELECT *
FROM ag_catalog.cypher('customer_graph', $$

  MATCH (c:Customer)
  WHERE c.payload.name = 'Customer 080'
  OPTIONAL MATCH (c)-[:RAISED_CASE]->(sc:SupportCase)
  WITH c, sc,
       CASE
         WHEN sc IS NOT NULL AND NOT (coalesce(sc.payload.status, '') IN ['Closed','Resolved','Canceled'])
         THEN true ELSE false
       END AS is_open
  WITH c, sc, is_open,
       collect(
         CASE WHEN is_open THEN {
           case_id: coalesce(sc.payload.id, ''),
           status: coalesce(sc.payload.status, ''),
           priority: coalesce(sc.payload.priority, ''),
           escalation_level: coalesce(sc.payload.escalation_level, 0),
           sla_breached: coalesce(sc.payload.sla_breached, false),
           opened_at: coalesce(sc.payload.opened_at, ''),
           last_updated_at: coalesce(sc.payload.last_updated_at, ''),
           product_area: coalesce(sc.p

In [ ]:
user_input = """

what are the top 3 products used by 'Customer 080'

"""
cypher_query = generate_cypher_query(user_input)



rows = await helper.query_using_sql_cypher(cypher_query)
print(rows)

In [12]:
user_input = """

I’m going on a sales call with customer 'Customer 080'” → Provide a consolidated customer insight including:

- Opportunities for upsell or cross-sell

"""
cypher_query = generate_cypher_query(user_input)



rows = await helper.query_using_sql_cypher(cypher_query)
print(rows)

Executing query:
 SELECT *
FROM ag_catalog.cypher('customer_graph', $$

  MATCH (c:Customer)
  WHERE c.payload.name = 'Customer 080'

  OPTIONAL MATCH (c)-[:RAISED_CASE]->(sc:SupportCase)
  WITH c,
       collect(CASE WHEN sc IS NOT NULL AND (sc.payload.status = 'Open' OR sc.payload.status = 'Pending' OR sc.payload.status = 'In Progress' OR sc.payload.status = 'Escalated') THEN {
         case_id: sc.payload.id,
         status: sc.payload.status,
         priority: sc.payload.priority,
         opened_at: sc.payload.opened_at,
         last_updated_at: sc.payload.last_updated_at,
         escalation_level: sc.payload.escalation_level,
         sla_breached: coalesce(sc.payload.sla_breached, false),
         product_area: sc.payload.product_area,
         subject: sc.payload.subject,
         tags: sc.payload.tags
       } ELSE NULL END) AS open_cases_tmp,
       sum(CASE WHEN sc IS NOT NULL AND (sc.payload.status = 'Open' OR sc.payload.status = 'Pending' OR sc.payload.status = 'In Pro